In [1]:
#Imports
import pandas as pd
import os
import tensorflow as tf

import wandb
from wandb.keras import WandbCallback
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint

#import sys  
#sys.path.append("../")  
#from utils.modelgenerator import *
#from utils.modelhandler import *
#from utils.datahandler import *

In [2]:
#Helper functions
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import time

#min_max_scaling
#Sclaes all columns of the dataframe df to the rang (0,1)
def min_max_scaling(df): #normailizing
    #Min Max Sclaing
    col_names = df.columns
    features = df[col_names]
    scaler = MinMaxScaler().fit(features.values)
    features = scaler.transform(features.values)
    df_scaled = pd.DataFrame(features, columns = col_names, index=df.index)
    return df_scaled

#create_sequences
#Split the dataframe into datasets with sequences of lngth=Sequence_length
def create_sequences(df, sequence_length):
    sequences = []
    for i in range(len(df) - sequence_length + 1):
        sequence = df.iloc[i:i+sequence_length, :]  # Take all columns
        sequences.append(sequence.values)
    return np.array(sequences)

#prepare_data
# Split each sequence into X (features) and Y (labels). 
# The label Y must be the FIRST column! The last batch is discarded, when < batch_size
def prepare_data(sequences, batch_size):
    X = sequences[:, :-1, :].astype('float32') #For all sequences, Exclude last row of the sequence, take all columns
    y = sequences[:, -1, 0].astype('float32') #For all sequences, Take the last row of the sequence, take the first column

    #As some models need to reshape the inputs, the correct batch_size is important
    #Adjust the dataset_size to be divisible by batch_size by discarding the remaining data points not fitting a complete batch.
    num_batches = len(X) // batch_size
    adjusted_X = X[:num_batches * batch_size]
    adjusted_y = y[:num_batches * batch_size]

    return adjusted_X, adjusted_y

class TimingCallback(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.start_time = time.time()
        self.epoch_times = []

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        epoch_end_time = time.time()
        epoch_time = epoch_end_time - self.epoch_start_time
        self.epoch_times.append(epoch_time)

    def get_training_times_df(self):
        total_training_time = time.time() - self.start_time
        average_epoch_times = [sum(self.epoch_times[:i+1]) / (i + 1) for i in range(len(self.epoch_times))]
        data = {
            'Epoch': list(range(1, len(self.epoch_times) + 1)),
            'Epoch Train_time': self.epoch_times,
            'Epoch Avg Train_time': average_epoch_times,
            'Total Training Time': total_training_time
        }
        return pd.DataFrame(data)
    

class CustomCallback(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.start_time = time.time()
        self.epoch_times = []
        self.losses = {
            'epoch': [],
            'train_loss': [],
            'val_loss': [],
            'test_loss': []
        }

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        epoch_end_time = time.time()
        epoch_time = epoch_end_time - self.epoch_start_time
        self.epoch_times.append(epoch_time)

        self.losses['epoch'].append(epoch)
        self.losses['train_loss'].append(logs['loss'])
        self.losses['val_loss'].append(logs['val_loss'])

    def on_test_end(self, logs=None):
        self.losses['test_loss'].append(logs['loss'])

    def get_loss_df(self):
        total_training_time = time.time() - self.start_time
        average_epoch_times = [sum(self.epoch_times[:i+1]) / (i + 1) for i in range(len(self.epoch_times))]
        self.losses['avg_epoch_time'] = average_epoch_times
        self.losses['total_training_time'] = total_training_time
        return pd.DataFrame(self.losses)

In [3]:
#Helper functions for models
from keras import layers, models

def build_dense_model(X_train, horizon, num_layers, units, batch_size):

    input_data = layers.Input(shape=(X_train.shape[1], X_train.shape[2]), batch_size=batch_size) 
    x =  layers.Dense(units, activation='relu')(input_data)
    for _ in range(num_layers-1):
        x = layers.Dense(units, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Flatten()(x)
    output = layers.Dense(horizon)(x) 

    dense_model = tf.keras.Model(inputs=input_data, outputs=output, name="Dense_model")

    return dense_model

#This method compiles the model using Adam optimizer, fits the model, and evaluates it
def compile_fit_evaluate_model(model, loss, metrics, X_train, y_train, max_epochs, batch_size, X_val, y_val, X_test, y_test, callbacks, wandb, user= "", hyper="", optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)):
    #Compile the model
    model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

    # Train the model
    history = model.fit(X_train, y_train, epochs=max_epochs, batch_size=batch_size, validation_data=(X_val, y_val), callbacks=callbacks, verbose=0,)
    #model = tf.keras.models.load_model('models/best_model.h5')
    #Evaluate the model on test dataset
    test_loss = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)

    train_times = callbacks[1].get_training_times_df()
    total_train_time = train_times["Total Training Time"][0]
    avg_time_epoch = train_times["Epoch Avg Train_time"].iloc[-1]

    model_user_result = pd.DataFrame(
        data=[[user, hyper, total_train_time, avg_time_epoch, test_loss[0], test_loss[1], test_loss[2], test_loss[3]]], 
        columns=["user", "architecture", "train_time", "avg_time_epoch", "mse", "rmse", "mape", "mae"]
    )

    return history, model_user_result

In [4]:
#Get data 
cwd = os.path.normpath(os.path.dirname(os.path.dirname(os.getcwd())))
df = pd.read_csv(cwd+'/data/2feature_engineering_data/df_with_final_features.csv', index_col='Date') #df = pd.read_csv('user5.csv')
df.index = pd.to_datetime(df.index)
#df = df[['User5', 'temp', 'rhum']]
df.fillna(0, inplace=True)

df_array = []
for idx in range(1):
    df_array.append(df[[f'User{idx+1}', 'temp', 'rhum', 'wspd', 'PC1', 'hour sin', 'hour cos', f'User{idx+1}_lag_24hrs']])

In [5]:
config = {
    "sequence_length": 25,
    "batch_size": 16,
    "num_features": df_array[0].shape[1],
    "horizon": 1,
    "max_epochs": 100,
    "dense_architecture": "L3_U16",
    "dense_layers": 3,
    "dense_units": 16,
    "loss": 'mean_squared_error',
    #"metric_rmse": tf.keras.losses.RootMeanSquaredError(),
    "metric_mape": 'mean_absolute_percentage_error',
    "metric_mae": 'mean_absolute_error',

}

#if wandb.run == None:
#    wandb.init(project="test-wandb-tracking")
#    wandb.config.update(config)
#else:
#    print("Already running")

#loss = tf.keras.losses.MeanSquaredError()
#metrics=[
#    tf.keras.metrics.RootMeanSquaredError(), 
#    tf.keras.metrics.MeanAbsolutePercentageError(),
#    tf.keras.metrics.MeanAbsoluteError(),
#]

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=10,mode='min')
timing_callback = TimingCallback()
custom_callback = CustomCallback()
#model_checkpoint = ModelCheckpoint('models/best_model.h5', save_best_only=True, monitor='val_loss', mode='min')

In [6]:
X_train, y_train, X_val, y_val, X_test, y_test = {}, {}, {}, {}, {}, {}

#Create Train, Validation and Test datasets
for idx, df in enumerate(df_array):
    n = len(df)
    train_df = df[0:int(n*0.7)]
    val_df = df[int(n*0.7):int(n*0.9)]
    test_df = df[int(n*0.9):]

    # Min max sclaing
    train_df = min_max_scaling(train_df)
    val_df = min_max_scaling(val_df)
    test_df = min_max_scaling(test_df)

    # Sequencing
    train_sequences = create_sequences(train_df, config['sequence_length'])
    val_sequences = create_sequences(val_df, config['sequence_length'])
    test_sequences = create_sequences(test_df, config['sequence_length'])

    #Split into feature and label
    X_train[f'user{idx+1}'], y_train[f'user{idx+1}'] = prepare_data(train_sequences, config['batch_size'])
    X_val[f'user{idx+1}'], y_val[f'user{idx+1}'] = prepare_data(val_sequences, config['batch_size'])
    X_test[f'user{idx+1}'], y_test[f'user{idx+1}'] = prepare_data(test_sequences, config['batch_size'])

In [7]:
dense_results = pd.DataFrame(columns=['architecture', 'train_time', 'avg_time_epoch', 'mse','mse_std', 'rmse','rmse_std','mape','mape_std','mae','mae_std'])

In [8]:
#Dense 1 -------------------------------------------------------------
dense_all_results = pd.DataFrame(columns=["user", "architecture", "train_time", "avg_time_epoch", "mse", "rmse", "mape", "mae"])
os.environ["WANDB_SILENT"] = "true"

#For each of the 3 user
for idx in range(len(df_array)):
    print("User: ", idx+1)
    for round in range(3):
        
        if wandb.run == None:
            wandb.init(
                project="test-wandb-tracking",
                name=f'userX{idx+1}_rd{round}'
                )
            wandb.config.update(config)
        else:
            print("Already running")

        #print("Round: ", round)
        dense_model = build_dense_model(
            X_train[f'user{idx+1}'], 
            wandb.config.horizon, 
            num_layers=wandb.config.dense_layers, 
            units=wandb.config.dense_units, 
            batch_size=wandb.config.batch_size
            )
                
        #Compile the model
        dense_model.compile(
            loss=wandb.config.loss, 
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
            metrics=[ wandb.config.metric_mape, wandb.config.metric_mae] #Double metric mae try RMSE
            )

          
        wandb_callbacks = [
            WandbMetricsLogger(
                log_freq='epoch',#("epoch", "batch", or int) if "epoch", logs metrics at the end of each epoch. If "batch", logs metrics at the end of each batch. If an integer, logs metrics at the end of that many batches. Defaults to "epoch".
            ),
            WandbModelCheckpoint(
                filepath="models/dense_model_{epoch:02d}",
                monitor='val_loss',
                verbose=0,
                save_best_only=False,
                save_weights_only=False,
                mode='min'
                )
        ]
        callbacks=[early_stopping, timing_callback, custom_callback, wandb_callbacks]
        
        # Train the model
        history = dense_model.fit(
            X_train[f'user{idx+1}'], 
            y_train[f'user{idx+1}'], 
            epochs=wandb.config.max_epochs, 
            batch_size=wandb.config.batch_size,
            validation_data=(X_val[f'user{idx+1}'], y_val[f'user{idx+1}']), 
            callbacks=callbacks, 
            verbose=0
            )
        #model = tf.keras.models.load_model('models/best_model.h5')
        #Evaluate the model on test dataset
        test_loss = dense_model.evaluate(
            X_test[f'user{idx+1}'], 
            y_test[f'user{idx+1}'], 
            batch_size=wandb.config.batch_size, 
            verbose=0
            )

        train_times = callbacks[1].get_training_times_df()
        total_train_time = train_times["Total Training Time"][0]
        avg_time_epoch = train_times["Epoch Avg Train_time"].iloc[-1]

        model_user_result = pd.DataFrame(
            data=[[f'user{idx+1}', wandb.config.dense_architecture, total_train_time, avg_time_epoch, test_loss[0], test_loss[0], test_loss[1], test_loss[2]]], #CHANGE!
            columns=["user", "architecture", "train_time", "avg_time_epoch", "mse", "rmse", "mape", "mae"]
        )
        wandb.finish()
        # Add the 'architecture' column from dense_user_results to dense_results
        #dense_all_results = pd.merge(dense_all_results, model_user_result, how='outer') 
        #dense_model.save(os.path.join(wandb.run.dir, "model.h5"))
        #wandb.save('model.h5')
        #wandb.finish()  

    #dense_model.save(cwd + f"/models/Local_learning/Dense/{dense_architecture}/User{idx}")
    print("Saved Soft_Dense_MoE")  



User:  1



INFO:tensorflow:Assets written to: models\dense_model_01\assets


INFO:tensorflow:Assets written to: models\dense_model_01\assets


INFO:tensorflow:Assets written to: models\dense_model_02\assets


INFO:tensorflow:Assets written to: models\dense_model_02\assets


INFO:tensorflow:Assets written to: models\dense_model_03\assets


...

INFO:tensorflow:Assets written to: models\dense_model_40\assets


INFO:tensorflow:Assets written to: models\dense_model_41\assets


INFO:tensorflow:Assets written to: models\dense_model_41\assets


Problem at: c:\Users\rs1044\Documents\GitHub\Mixture-of-Experts-based-Federated-Learning-for-Energy-Forecasting\.venv\Lib\site-packages\wandb\sdk\wandb_init.py 848 getcaller


KeyboardInterrupt: 

In [9]:
if wandb.run == None:
    wandb.init(project="test-wandb-tracking")
    wandb.config.update(config)
else:
    print("Already running")
    
for idx in range(len(df_array)):
    new_row = {
        'architecture': wandb.config.dense_architecture,
        'train_time': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["train_time"].mean(), 
        'avg_time_epoch' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["avg_time_epoch"].mean(),
        'mse': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mse"].mean(),
        'mse_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mse"].std(),
        'rmse': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["rmse"].mean(),
        'rmse_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["rmse"].std(),
        'mape': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mape"].mean(),
        'mape_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mape"].std(),
        'mae': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mae"].mean(),
        'mae_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mae"].std(),
    }
    dense_results.loc[len(dense_results)] = new_row

wandb.finish()  

In [11]:
#dense_results.to_csv(f'../../evaluations/Test_Datastream_Dense.csv')
dense_results

,architecture,train_time,avg_time_epoch,mse,mse_std,rmse,rmse_std,mape,mape_std,mae,mae_std
0,L3_U16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
